In [1]:
import ccxt
from datetime import datetime as datetime,timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import warnings
import time
import pytz
import math
import schedule

warnings.filterwarnings('ignore')

apiKey = 'apikey'
secretKey = 'secret key api'
# !pip install -U vectorbt
exchange = ccxt.binance({'apiKey':apiKey,'secret':secretKey,'enableRateLimit':True})

In [3]:
#ticker_list = ['BTC/BUSD','ETH/BUSD','BNB/BUSD']
ticker_list = ['ETH/BUSD','BNB/BUSD']

value_ = [10,10,10] # $ Usd value /symbols


bot_setting= dict(zip(ticker_list,value_))
bot_setting

{'ETH/BUSD': 10, 'BNB/BUSD': 10}

In [4]:
def get_data(symbol,timeframe='1m',limit=500):
    bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    # bars is data from ccxt 
    df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    # datamodeling bars to dataframe 
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    # set timestamp format
    df = df.set_index('timestamp')
    # setindex to timestamp
    return df

In [5]:
def download_data(ticker_list,timeframe):
    coin_dict = {}
    for ticker in ticker_list:
        df= get_data(ticker,timeframe=timeframe)
        coin_dict[ticker] = df
        df = pd.concat(coin_dict,axis=0)
    return df

In [6]:
# Loop to creates indicator values
def get_signal(dfbackup):
    df['ema'] =  df.ta.ema(length=3)
    df['sma'] = df.ta.sma(length=3)
    df['trend'] = df['sma'] > df['ema'] 
    df['signal'] =    np.where((df['trend']==True) & (df['trend'].shift()==False), 'buy', np.NaN)
    df['signal'] =    np.where((df['trend']==False) & (df['trend'].shift()==True), 'sell', df['signal'])
    
    return df

In [19]:
def get_wallet():
    res = exchange.fetch_balance()
    wallet= pd.DataFrame(res['info']['balances'])
    wallet['free'] =wallet['free'].astype(float)
    wallet =wallet.sort_values('free',ascending=False)
    return wallet

def get_time():
    time_str=datetime.now(pytz.timezone('Asia/Bangkok')).strftime('%Y-%m-%d' ' %H:%M:%S')
    return time_str

def sell_market(symbol,size_=0):
    try:
        wallet= get_wallet()
        name = symbol.split('/')[0]
        if size_== 0 :
            size_= wallet[wallet['asset']== name ]['free'].values[0] # check values in wallet 
        res=exchange.create_order(symbol,'market','sell',size_) # sell 
        print(f'Sell {symbol}  Market order {size_} unit ' )
    except Exception as e :
        print(get_time(),' : ',str(e))
        res =[]
    return res

def buy_market(symbol,size_):
    try:
        res= exchange.create_order(symbol,'market','buy',size_)
        print(f'Buy {symbol}   Market order {size_} unit ' )

    except Exception as e:
        print(get_time(),' : ',str(e))
        res= []
    except ccxt.InvalidOrder as e:
        last,digit,min_size=get_detail()
        size_ = size_ + min_size 
        res= exchange.create_order(symbol,'market','buy',size_)
        print(f'Buy Market order {symbol} {size_} unit ' )
    return res

In [10]:
def roundup(x,increment=5):
     # step_incre,ment 
    return int(math.ceil(x / increment)) * increment

def get_min_size(symbol):
    res= exchange.market(symbol)
    min_size= float(res['info']['filters'][1]['minQty'])
    return min_size

def get_digit(symbol):
    price_res = exchange.fetch_ticker(symbol)
    last = price_res['last']
    min_size = get_min_size(symbol)
    # min size from exchange api 
    digit_ =  float(min_size)+1
    # note +1 cuase if digit morethan 7 will not cannot split must plus 1can split in next line
    digit= len(str(digit_).split('.')[1])
    #print(price_res['close'],'digit :',digit,min_size)
    return digit

In [49]:
def get_action(signal):
    for ticker in ticker_list:
        last_sig =signal[ticker][-1]
        fix_value = bot_setting[ticker]
        price= exchange.fetch_ticker(ticker)['last']
        min_size= get_min_size(ticker)
        ############## AMOUNT CHECK SIZE RESIZE TO Avaiable Trades
        amount = fix_value / price
        amount_ = roundup(amount,min_size)
        amount_ = round(amount_,get_digit(ticker))
        value_ = round(amount_  * price,2)
        time_= get_time()
        if last_sig == 'buy':
            print(f'[{time_}] Buy {ticker} {price} {amount_} {value_}')
            buy_log.append('Buy '+ str(ticker) + ' at : ' + str(datetime.now(pytz.timezone('Asia/Bangkok')).strftime('%Y-%m-%d' ' %H:%M:%S')))
            buy_res= buy_market(ticker,amount_)
            if buy_res != None:
                print('Buy Complete :')
            else:
                print(f'Buy Not Complete : {amount_}')
                print('#'*50)
        elif last_sig == 'sell':
            print(f'[{time_}] Sell {ticker} {price} {amount_} {value_}')
            sell_log.append('sell '+ str(ticker) + ' at : ' + str(datetime.now(pytz.timezone('Asia/Bangkok')).strftime('%Y-%m-%d' ' %H:%M:%S')))
            sell_res = sell_market(ticker,amount_)
            if sell_res != None:
                print('Sell Comeplete:')
            else:
                print(f'Sell Not Complete : {amount_}')
                print('#'*50)
                
        else:
            print(ticker,'No signal',)
            print('-'*50)
       # print(ticker)
      #  print(signal[ticker][-1])
       # print(f'minsize :{min_size}')
       # print(bot_setting[ticker])
      #  print(amount)
     # print(amount_)       
      #  print(value_)

In [50]:
# debug session

df= download_data(ticker_list,'1m')
stg = get_signal(df)
signal = stg['signal'].unstack(0)



In [51]:
# debug session
signal

,ETH/BUSD,BNB/BUSD
timestamp,,
2023-01-15 09:38:00,nan,nan
2023-01-15 09:39:00,nan,nan
2023-01-15 09:40:00,nan,sell
2023-01-15 09:41:00,buy,nan
2023-01-15 09:42:00,nan,nan
...,...,...
2023-01-15 17:53:00,nan,nan
2023-01-15 17:54:00,nan,sell
2023-01-15 17:55:00,nan,buy


In [45]:
# debug session
get_action(signal)

                    ETH/BUSD BNB/BUSD
timestamp                            
2023-01-15 09:31:00      nan      buy
2023-01-15 09:32:00      nan      nan
2023-01-15 09:33:00      nan     sell
2023-01-15 09:34:00      nan      nan
2023-01-15 09:35:00      nan      nan
...                      ...      ...
2023-01-15 17:46:00      buy      nan
2023-01-15 17:47:00      nan      buy
2023-01-15 17:48:00      nan      nan
2023-01-15 17:49:00      nan      nan
2023-01-15 17:50:00     sell     sell

[500 rows x 2 columns]
[2023-01-16 00:50:39] Sell ETH/BUSD 1551.66 0.0065 10.09
2023-01-16 00:50:40  :  binance Account has insufficient balance for requested action.
Sell Comeplete:
[2023-01-16 00:50:40] Sell BNB/BUSD 301.3 0.034 10.24
Sell BNB/BUSD  Market order 0.034 unit 
Sell Comeplete:


In [54]:
    buy_log = []
    sell_log = []
    coin_dict = {}
while True: 
        time.sleep(60-time.time()%60)
        df= download_data(ticker_list,'1m')
        stg = get_signal(df)
        signal = stg['signal'].unstack(0)

        get_action(signal)

[2023-01-16 00:58:00] Sell ETH/BUSD 1552.26 0.0065 10.09
2023-01-16 00:58:01  :  binance Account has insufficient balance for requested action.
Sell Comeplete:
BNB/BUSD No signal
--------------------------------------------------
[2023-01-16 00:59:00] Buy ETH/BUSD 1552.76 0.0065 10.09
Buy ETH/BUSD   Market order 0.0065 unit 
Buy Complete :
[2023-01-16 00:59:01] Buy BNB/BUSD 301.2 0.034 10.24
Buy BNB/BUSD   Market order 0.034 unit 
Buy Complete :
ETH/BUSD No signal
--------------------------------------------------
BNB/BUSD No signal
--------------------------------------------------
ETH/BUSD No signal
--------------------------------------------------
BNB/BUSD No signal
--------------------------------------------------
ETH/BUSD No signal
--------------------------------------------------
[2023-01-16 01:02:01] Sell BNB/BUSD 300.8 0.034 10.23
Sell BNB/BUSD  Market order 0.034 unit 
Sell Comeplete:
ETH/BUSD No signal
--------------------------------------------------
BNB/BUSD No signal
-

KeyboardInterrupt: 